In [1]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_curve, confusion_matrix, auc,classification_report
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline


import numpy as np


In [2]:
import pandas as pd

# Carregue o arquivo Excel
caminho_arquivo = '[Dados] Case - Data - Latam.xlsx'  # Substitua pelo caminho do seu arquivo
dados_excel = pd.read_excel(caminho_arquivo)

# Visualize os primeiros registros para garantir que o arquivo foi lido corretamente
print(dados_excel.head())


     País          Cidade    Tipo de Produto      Origem do Lead        Mês  \
0  Brazil  Belo Horizonte  WeWork All Access  E-Commerce Closing 2021-01-08   
1  Brazil  Belo Horizonte  WeWork All Access  E-Commerce Closing 2021-01-09   
2  Brazil  Belo Horizonte  WeWork All Access  E-Commerce Closing 2021-01-10   
3  Brazil  Belo Horizonte  WeWork All Access  E-Commerce Closing 2021-01-11   
4  Brazil  Belo Horizonte  WeWork All Access  E-Commerce Closing 2021-01-12   

   Novos Contratos  Contratos Totais  Contratos Encerrados  Novas Contas  \
0                4                 4                     0             4   
1               28                29                     3            28   
2               12                20                    21            12   
3               16                24                    12            16   
4                8                11                    21             8   

   Total de Contas  Contas Encerradas  Novos Membros  Total de Membr

In [3]:
dados_excel.columns

Index(['País', 'Cidade', 'Tipo de Produto', 'Origem do Lead', 'Mês',
       'Novos Contratos', 'Contratos Totais', 'Contratos Encerrados',
       'Novas Contas', 'Total de Contas', 'Contas Encerradas', 'Novos Membros',
       'Total de Membros', 'Membros Inativos'],
      dtype='object')

In [4]:
novas_contas_membros_por_mes = dados_excel.groupby('Mês')[['Novas Contas', 'Novos Membros']].sum()


In [5]:
novas_contas_membros_por_mes

,Novas Contas,Novos Membros
Mês,,
2021-01-01,12,218
2021-01-02,6,82
2021-01-03,50,66
2021-01-04,54,92
2021-01-05,188,658
2021-01-06,268,3366
2021-01-07,460,782
2021-01-08,900,1744
2021-01-09,1288,4706


In [6]:
distribuicao_geografica = dados_excel.groupby(['País', 'Cidade']).size().reset_index(name='Número de Usuários')


In [8]:
distribuicao_geografica_ordenada = distribuicao_geografica.sort_values(by='Número de Usuários', ascending=False)


In [9]:
distribuicao_geografica_ordenada 

,País,Cidade,Número de Usuários
6,Brazil,São Paulo,34
12,Mexico,Ciudad de México,34
4,Brazil,Rio de Janeiro,26
0,Argentina,Buenos Aires,23
14,Mexico,Monterrey,23
16,Mexico,Zapopan,22
1,Brazil,Belo Horizonte,21
9,Colombia,Bogotá,21
13,Mexico,Guadalajara,20
5,Brazil,São Bernardo do Campo,19


In [10]:
tipo_produto_popular = dados_excel.groupby('Tipo de Produto').size().reset_index(name='Número de Contratos')


In [11]:
tipo_produto_popular

,Tipo de Produto,Número de Contratos
0,WeWork All Access,312
1,WeWork All Access - Pay Per Use,17


In [12]:
# Calcula a taxa de crescimento das contas por país e cidade
taxa_crescimento_contas_por_local = dados_excel.groupby(['País', 'Cidade'])['Total de Contas'].pct_change() * 100

# Cria um novo DataFrame com o país, cidade e taxa de crescimento das contas
df_taxa_crescimento_contas_por_local = pd.DataFrame({
    'País': dados_excel['País'],
    'Cidade': dados_excel['Cidade'],
    'Taxa de Crescimento das Contas': taxa_crescimento_contas_por_local
})

# Obtém o top 10 dos resultados ordenados
top_10_taxa_crescimento_contas_por_local = df_taxa_crescimento_contas_por_local.nlargest(10, 'Taxa de Crescimento das Contas')

# Exibe o top 10 dos resultados
print(top_10_taxa_crescimento_contas_por_local)



         País                 Cidade  Taxa de Crescimento das Contas
100     Chile      Santiago de Chile                             inf
137    Mexico      Centro, Monterrey                             inf
178    Mexico                Zapopan                             inf
223    Brazil         Rio de Janeiro                             inf
230    Brazil  São Bernardo do Campo                             inf
305    Mexico            Guadalajara                             inf
28     Mexico       Ciudad de México                          1100.0
266  Colombia                 Bogotá                          1000.0
88     Brazil              São Paulo                           900.0
234    Brazil              São Paulo                           900.0


In [13]:
taxa_rescisao_contas = (dados_excel['Contas Encerradas'] / dados_excel['Total de Contas']) * 100

In [14]:
taxa_rescisao_contas

0        0.000000
1       10.344828
2      105.000000
3       50.000000
4      190.909091
          ...    
324      0.000000
325    120.000000
326    600.000000
327     44.444444
328    225.000000
Length: 329, dtype: float64

In [15]:

dados_excel['Taxa de Inatividade de Membros'] = (dados_excel['Membros Inativos'] / dados_excel['Total de Membros']) * 100


df_top_10_inatividade = dados_excel.sort_values(by='Taxa de Inatividade de Membros', ascending=False).head(10)

# Exiba o top 10 das taxas de inatividade de membros
print(df_top_10_inatividade[['País', 'Cidade', 'Taxa de Inatividade de Membros']])


          País                 Cidade  Taxa de Inatividade de Membros
216     Brazil         Rio de Janeiro                             inf
304     Mexico            Guadalajara                             inf
136     Mexico      Centro, Monterrey                             inf
295     Mexico       Ciudad de México                     1166.283525
56   Argentina           Buenos Aires                      900.000000
59      Brazil         Belo Horizonte                      450.000000
16      Brazil  São Bernardo do Campo                      400.000000
204     Brazil         Belo Horizonte                      400.000000
328     Mexico                Zapopan                      300.000000
114   Colombia           Barranquilla                      300.000000


In [16]:
taxa_conversao_origem_lead = (dados_excel.groupby('Origem do Lead')['Contratos Totais'].sum() / dados_excel['Contratos Totais'].sum()) * 100


In [17]:
taxa_conversao_origem_lead

Origem do Lead
E-Commerce Closing      21.751068
E-Commerce Lead         32.207138
Sales Lead & Closing    46.041794
Name: Contratos Totais, dtype: float64

In [18]:

distribuicao_origem_lead = (dados_excel.groupby('Origem do Lead').size() / len(dados_excel)) * 100


In [19]:
distribuicao_origem_lead

Origem do Lead
E-Commerce Closing      13.981763
E-Commerce Lead         42.553191
Sales Lead & Closing    43.465046
dtype: float64

In [20]:

taxa_conversao_tipo_produto = (dados_excel.groupby('Tipo de Produto')['Contratos Totais'].sum() / dados_excel['Contratos Totais'].sum()) * 100


In [21]:
taxa_conversao_tipo_produto

Tipo de Produto
WeWork All Access                  99.710189
WeWork All Access - Pay Per Use     0.289811
Name: Contratos Totais, dtype: float64